In [7]:
!pip3 install pandasql
import os 
import pandas as pd 
from pandasql import sqldf
# pandasql을 전역 환경으로 지정
mysql = lambda q: sqldf(q, globals())

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [ ]:
Employees_dict = {
    "사원_ID" : [1,2,3,4],
    "사원_이름" : ['김철수', '박영희', '최유리', '이민준'],
    "부서_ID" : [101, 102, 101, None]
}

Departments_dict = {
        "부서_ID" : [101, 102, 103],
        "부서_이름" : ['인사팀','개발팀','마케팅팀']    
}

Employees = pd.DataFrame(Employees_dict)
Departments = pd.DataFrame(Departments_dict)



In [5]:
Employees.head()

,사원_ID,사원_이름,부서_ID
0,1,김철수,101.0
1,2,박영희,102.0
2,3,최유리,101.0
3,4,이민준,NaN


In [6]:
Departments.head()

,부서_ID,부서_이름
0,101,인사팀
1,102,개발팀
2,103,마케팅팀


In [8]:
mysql("""
      SELECT e.사원_이름, d.부서_이름
      FROM Employees AS e
      JOIN Departments AS d
      ON e.부서_ID = d.부서_ID
      """)

,사원_이름,부서_이름
0,김철수,인사팀
1,박영희,개발팀
2,최유리,인사팀


In [9]:
mysql("""
      SELECT e.사원_이름, d.부서_이름
      FROM Employees AS e
      LEFT JOIN Departments AS d
      ON e.부서_ID = d.부서_ID
      """)

,사원_이름,부서_이름
0,김철수,인사팀
1,박영희,개발팀
2,최유리,인사팀
3,이민준,None


In [10]:
# 'Departments' (부서) 데이터프레임 생성
departments_data = {
    'dept_id': ['D101', 'D102', 'D103'],
    'dept_name': ['개발팀', '기획팀', '마케팅팀']
}
departments = pd.DataFrame(departments_data)


# 'Employees' (직원) 데이터프레임 생성
employees_data = {
    'emp_id': [1, 2, 3, 4, 5, 6, 7],
    'emp_name': ['김개발', '박기획', '이마케', '최코딩', '정디자인', '한분석', '윤홍보'],
    'dept_id': ['D101', 'D102', 'D103', 'D101', 'D102', 'D101', 'D103'],
    'position': ['팀장', '팀장', '사원', '사원', '대리', '대리', '대리'],
    'salary': [9000, 8500, 5500, 6000, 7200, 7500, 6800],
    'hire_date': ['2020-01-15', '2019-08-01', '2022-03-10', '2023-05-20', '2021-11-22', '2021-02-18', '2022-09-30']
}
employees = pd.DataFrame(employees_data)

In [13]:
mysql("""
      SELECT dept_id AS "부서", COUNT(emp_name) AS "직원 수"
      , AVG(salary) AS "평균 연봉"
      FROM employees
      GROUP BY 1
      HAVING AVG(salary) >= 7500 
      """)

,부서,직원 수,평균 연봉
0,D101,3,7500.0
1,D102,2,7850.0


In [15]:
mysql("""
      SELECT e.emp_name AS "직원명", d.dept_name AS "부서명" 
      FROM employees AS e
      JOIN departments AS d
      USING (dept_id)
      """)

,직원명,부서명
0,김개발,개발팀
1,박기획,기획팀
2,이마케,마케팅팀
3,최코딩,개발팀
4,정디자인,기획팀
5,한분석,개발팀
6,윤홍보,마케팅팀


In [ ]:
mysql("""
      SELECT *
      FROM employees
      WHERE dept_id = (SELECT dept_id 
                       FROM departments 
                       WHERE dept_name = '개발팀') 
      """)

,emp_id,emp_name,dept_id,position,salary,hire_date
0,1,김개발,D101,팀장,9000,2020-01-15
1,4,최코딩,D101,사원,6000,2023-05-20
2,6,한분석,D101,대리,7500,2021-02-18


In [30]:
mysql("""
      SELECT emp_name AS '직원' , salary AS '연봉', 
      (SELECT AVG(salary) FROM employees) AS '전체 직원 평균 연봉'
      FROM employees;
      """)

,직원,연봉,전체 직원 평균 연봉
0,김개발,9000,7214.285714
1,박기획,8500,7214.285714
2,이마케,5500,7214.285714
3,최코딩,6000,7214.285714
4,정디자인,7200,7214.285714
5,한분석,7500,7214.285714
6,윤홍보,6800,7214.285714


In [33]:
mysql("""
      SELECT d.dept_name AS "부서명" , COUNT(e.dept_id) AS "직원수" 
      FROM departments AS d
      JOIN employees AS e
      USING (dept_id)
      GROUP BY dept_name
      ORDER BY COUNT(*) DESC
      """)

,부서명,직원수
0,개발팀,3
1,마케팅팀,2
2,기획팀,2


# 오늘의 퀴즈

## 1. "우리 쇼핑몰의 VIP 고객을 찾고 싶습니다. 
- 총 주문 금액(amount의 합계)이 100,000원 이상인 고객들의 이름(name)과 총 주문 금액을 출력해주세요."

    - customers 테이블: customerId, name

    - orders 테이블: orderId, customerId, amount (주문금액)

In [ ]:
mysql("""
     SELECT c.name AS "이름", SUM(o.amount) AS "총 주문 금액" 
     FROM customers AS c
     JOIN orders AS o
     ON c.customerId = o.customerId 
     GROUP BY c.name
     HAVING SUM(o.amount) >= 100000
      """)

# 2. 작년보다 더 잘 버는 직무 찾기
- 2020년(작년)의 전체 평균 연봉보다 더 높은 평균 연봉을 받는 2021년의 직무는 무엇이 있을까요?


[요구사항]

서브쿼리 사용: ds_jobs_2020 테이블을 사용하여 **2020년의 전체 평균 연봉(salary_in_usd)**을 구하세요.

메인 쿼리: ds_jobs_2021 테이블에서 직무(job_title)별 평균 연봉을 구하세요.

조건(HAVING): 2021년의 직무별 평균 연봉이 아까 구한 '2020년 전체 평균'보다 높은 직무만 필터링하세요.

정렬: 평균 연봉이 높은 순서대로 내림차순 정렬해주세요.

[출력컬럼]

job_title (직무명), avg_salary_2021 (2021년 해당 직무의 평균 연봉)


In [ ]:
mysql("""
      SELECT job_title AS "직무명", AVG(salary_in_usd) AS "직무별 평균 연봉"
      FROM ds_jobs_2021
      GROUP BY 1
      HAVING AVG(salary_in_usd) > (SELECT AVG(salary_in_usd) FROM ds_jobs_2020)
      ORDER BY AVG(salary_in_usd) DESC
      """)